# Capítulo 05 — Pré-Treinamento: O Nascimento da Inteligência

Neste capítulo, vamos tirar o nosso GPTMini da inércia. Vamos alimentá-lo com dados e observar o momento exato em que ele deixa de escolher letras aleatórias e começa a formar palavras e frases.

--- 
### 🏭 A Fábrica de Predição
O pré-treinamento ensina ao modelo a base da nossa língua. Ele aprende que depois de "o g" provavelmente vem um "a", formando "o ga...".

![Pipeline de Treinamento](./infograficos/01-pipeline-treinamento.png)

In [ ]:
# ============================================================
# Setup do repositório no Colab
# ============================================================
import os
import sys

REPO_NAME = "fazendo-um-llm-do-zero"

if 'google.colab' in str(get_ipython()):
    if not os.path.exists(REPO_NAME):
        !git clone https://github.com/vongrossi/{REPO_NAME}.git
    
    if os.path.exists(REPO_NAME) and os.getcwd().split('/')[-1] != REPO_NAME:
        os.chdir(REPO_NAME)

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

print("📂 Diretório atual:", os.getcwd())

In [ ]:
# Setup e Imports
import os, sys, torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from lib.gptmini import GPTConfig, GPTMini

device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)
print(f"✅ Oficina pronta! Usando: {device}")

## 1. O Combustível: Tokenização por Caracteres

Para nosso modelo "Mini", vamos tratar cada letra e sinal de pontuação como um token único.

In [ ]:
text = """
o gato subiu no telhado e pulou o muro.
o cachorro subiu no sofa e dormiu no tapete.
o gato dormiu no sofa enquanto o cachorro pulou o portao.
inteligencia artificial e o estudo de algoritmos que aprendem.
machine learning permite que sistemas aprendam padroes de dados.
""".strip().lower()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long).to(device)
print(f"Vocabulário: {''.join(chars)} | Tamanho: {vocab_size}")

## 2. Preparando a Janela de Contexto

O modelo olha para os últimos `context_size` caracteres para prever o próximo.

In [ ]:
context_size = 32
def get_batch(data, batch_size=16):
    ix = torch.randint(len(data) - context_size, (batch_size,))
    x = torch.stack([data[i:i+context_size] for i in ix])
    y = torch.stack([data[i+1:i+context_size+1] for i in ix])
    return x, y

xb, yb = get_batch(data)
print("Exemplo de Entrada (X):", decode(xb[0].tolist()))
print("Exemplo de Alvo (Y)   :", decode(yb[0].tolist()))

## 3. Treinamento: Da Aleatoriedade à Lógica

Vamos treinar o modelo e salvar o histórico de erro (Loss).

In [ ]:
config = GPTConfig(vocab_size=vocab_size, context_size=context_size, d_model=128, n_heads=4, n_layers=3)
model = GPTMini(config).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

loss_history = []
print("🔨 Treinando...")
for step in range(1001):
    xb, yb = get_batch(data)
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())
    if step % 200 == 0: print(f"Step {step:04d} | Loss: {loss.item():.4f}")

plt.figure(figsize=(8, 3))
plt.plot(loss_history, color='#1A73E8')
plt.title("Aprendizado do Modelo (Loss)")
plt.show()

## 4. Teste de Geração e Salvamento

Agora o modelo deve completar frases de forma minimamente coerente.

In [ ]:
@torch.no_grad()
def generate(model, start_str, max_new=50):
    model.eval()
    idx = torch.tensor(encode(start_str)).unsqueeze(0).to(device)
    for _ in range(max_new):
        logits, _ = model(idx[:, -context_size:])
        probs = F.softmax(logits[:, -1, :], dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat([idx, next_id], dim=1)
    return decode(idx[0].tolist())

print("✨ Resultado:", generate(model, "o gato "))

# SALVAMENTO CRUCIAL: Pesos + Dicionário
checkpoint = {
    "state_dict": model.state_dict(),
    "stoi": stoi,
    "itos": itos,
    "config": config
}
torch.save(checkpoint, "gpt_checkpoint.pt")
print("✅ Checkpoint completo salvo!")

## 🏁 Conclusão

Você acabou de ensinar um GPT a aprender linguagem. Viu como ele sai do caos aleatório para a ordem gramatical e como podemos controlar sua criatividade.

No próximo capítulo, vamos levar este modelo para a **especialização**, aprendendo como fazer o **Fine-Tuning** para tarefas específicas como classificação de sentimentos e detecção de spam.